爬蟲 需要的套件

In [1]:
#!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from urllib import request
import random

Redis 需要的套件

In [2]:
#!pip install redis
import redis
from hashlib import md5

In [3]:
class SimpleHash(object):
    def __init__(self, cap, seed):
        self.cap = cap
        self.seed = seed

    def hash(self, value):
        ret = 0
        for i in range(len(value)):
            ret += self.seed * ret + ord(value[i])
        return (self.cap - 1) & ret


class BloomFilter(object):
    def __init__(self, host='redis', port=6379, db=0, blockNum=1, key='bloomfilter'):
        """
        :param host: the host of Redis
        :param port: the port of Redis
        :param db: witch db in Redis
        :param blockNum: one blockNum for about 90,000,000; if you have more strings for filtering, increase it.
        :param key: the key's name in Redis
        """
        self.server = redis.Redis(host=host, port=port, db=db)
        self.bit_size = 1 << 31  # Redis的String类型最大容量为512M，现使用256M
        self.seeds = [5, 7, 11, 13, 31, 37, 61]
        self.key = key
        self.blockNum = blockNum
        self.hashfunc = []
        for seed in self.seeds:
            self.hashfunc.append(SimpleHash(self.bit_size, seed))

    def isContains(self, str_input):
        if not str_input:
            return False
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        ret = True
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            ret = ret & self.server.getbit(name, loc)
        return ret

    def insert(self, str_input):
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            self.server.setbit(name, loc, 1)

BloomFilter方法:處理資料去重複

In [4]:
bf = BloomFilter()
# url = "https://tw.news.appledaily.com/politics/realtime/20181125/1473059/"
# url = url.encode('utf-8')
# if bf.isContains(url):   # 判断字符串是否存在
#     print('url is exists!')
# else:
#     print('url is not exists!')
#     bf.insert(url)

In [5]:
#BloomFilter的key
bf.key

'bloomfilter'

In [6]:
pool = redis.ConnectionPool(host='redis', port=6379, db=0)
r = redis.StrictRedis(connection_pool=pool)

In [7]:
#搜尋redis裡面有哪些keys
keys = r.keys()
print(keys)

[b'bloomfilter0']


In [8]:
#刪除redis裡key為XXX
r.delete("bloomfilter0")

1

傳送log 需要的套件

In [9]:
#!pip install python-logstash
import logging
import logstash
import sys

In [10]:
host = "logstash"
crawler_logger = logging.getLogger('crawler_logger')
crawler_logger.setLevel(logging.INFO)

# TCP
crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

Kafka Producer 需要的套件

In [11]:
#!pip install kafka
from kafka import KafkaProducer
import sys
from kafka.errors import KafkaError

連線Kafka

In [12]:
producer = KafkaProducer(
    # Kafka集群在那裡?
    bootstrap_servers = ["kafka1:29092"],
    # 指定msgKey的序列化器, 若Key為None, 無法序列化, 透過producer直接給值
    key_serializer = str.encode,
    # 指定msgValue的序列化器
    value_serializer = lambda m: json.dumps(m).encode('utf-8')
)

TVBS 爬蟲方法

In [13]:
def tvbs_crawler2kafka():
    try:
        ###TVBS爬蟲開始###
        url = "https://news.tvbs.com.tw/politics/"
        response = urlopen(url)
        soup = BeautifulSoup(response)
        news_ul = soup.find_all("ul", id = "block_pc")[0]
        news_a = news_ul.find_all("a")

        news_no_string_list = []
        for a in news_a:
            news_no_string = a["data-news-id"]
            news_no_string_list.append(news_no_string)

        news_no_list = []
        for n in news_no_string_list:
            news_no_int = n.strip('\'')
            news_no_list.append(news_no_int)

        for news_no in news_no_list:

            print("新聞來源:", "TVBS新聞台")

            news_url = "https://news.tvbs.com.tw/politics/" + news_no
            print("新聞網址:", news_url)


            #判斷網址是否爬過
            bf = BloomFilter()
            # Unicode-objects must be encoded before hashing
            news_url_u = news_url.encode('utf-8')
            if bf.isContains(news_url_u):   # 判断字符串是否存在
                print('url is exists!')
                continue
            else:
                print('url is not exists!')
                bf.insert(news_url_u)


            req = requests.get(news_url)
            # requests如果找不到指定編碼，會猜測網頁編碼，有時會形成亂碼，故給指定編碼utf-8
            req.encoding = ("utf-8")
            soup = BeautifulSoup (req.text, 'html.parser')

            title = soup.find("h1", class_ = "margin_b20").text
            title = re.sub(r"\u3000", " ", title)
            title = re.sub(r"\xa0", " ", title)
            print("新聞標題:", title)

            date = soup.find("div", class_ = "icon_time time leftBox2").text
            print("發佈時間:", date)

            author = []
            author_text = soup.find("h4", class_ = "font_color5 leftBox1").text
            author_text = re.sub(r"攝影.*報導", "", author_text)
            author.append(author_text)
            print("作者:", author)

            content = soup.find("div", class_ = "h7 margin_b20").text
            content = content.replace("\t", "").replace("   ","").replace("\n", "").replace("（中央社）","")
            content = content.replace("最HOT話題在這！想跟上時事，快點我加入TVBS新聞LINE好友！ ","")
            content = content.replace("最HOT話題在這！想跟上時事，","").replace("快點我加入TVBS新聞LINE好友！ ","")
            content = content.replace("TVBS最新大數據分析","").replace("看完整內容快點我加入TVBS新聞LINE好友！","")
            print("內文:", content)

            keyword = soup.find("div", class_ = "adWords").text
            keyword = keyword.replace("\t", "")
            keyword = keyword.replace(",","")
            keyword = keyword.replace("\n", "").replace("編輯  ","").replace(" 報導","")
            keyword = keyword.replace("記者","").replace("      ",",")
            keyword = keyword.split(",")
            print("關鍵字", keyword)
            
            
            img = soup.find("div", class_ = "margin_b20")
            if not img == None:
                img_url = img.find('img')['src']
            else:
                img_url = ""
            print("圖片網址:", img_url)

            j = {"source": "TVBS新聞台" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                 "kw": keyword, "img_url": img_url}
            time.sleep(1)

            # TCP連線到logstash
            crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
            # 爬蟲成功傳送至Elasticsearch的log訊息
            crawler_logger.info('python-crawler-logstash: TVBS crawler success!')
            crawler_logger.handlers.clear()
        ###TVBS爬蟲結束###
    
            # 步驟2.指定想要發佈訊息的topic名稱
            topic_name = "politics_news"

            print("Now sending messages ...")
            # 步驟3.產生要發佈到Kafka的訊息
            # - 參數  # 1: topicName
            # - 參數  # 2: msgKey
            # - 參數  # 3: msgValue

            producer.send(topic = topic_name, key = "tvbs", value = j)
            print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: TVBS crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

SETN爬蟲方法

In [14]:
def setn_crawler2kafka():
    try:
        ###SETN爬蟲開始###
        for page in range(1, 5):
            page_url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=" + str (page)
            print ("Processing:", page_url)

            response = urlopen (page_url)
            bs = BeautifulSoup (response)
            titles = bs.find_all ("h3", "view-li-title")

            if len (titles) == 0:
                break

            for t in titles:

                print("新聞來源:", "SETN三立新聞網")

                news_url = "https://www.setn.com" + t.find ("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                req = requests.get (news_url)
                bs = BeautifulSoup (req.text, 'html.parser')

                title = bs.find('h1', class_ = "news-title-3").text
                title = re.sub(r"\u3000", " ", title)
                print("新聞標題:", title)

                date = bs.find("time", class_ = "page-date").text.strip()
                print("發佈時間:", date)

                author = []
                a_divs = bs.find("div", id="Content1")
                first_p = a_divs.find("p")
                author_text = first_p.text
                #找第一段有沒有含"／"，如果沒有給予空字串
                if author_text.find("／") == -1:
                    author_text = ""
                #如果作者是空字串，給予空list
                if author_text == "":
                    author = []
                else:
                    author.append(author_text)
                print("作者:", author)

                c_divs = bs.find('div', itemprop='articleBody')
                content_all = c_divs.find_all ("p")
                content = ""
                for c in content_all:
                    if c.attrs == {}:
                        content = content + c.text
                        #print(c.text)

                        c_divs = bs.find('div', itemprop='articleBody')
                content_all = c_divs.find_all ("p")
                content = ""
                for c in content_all:
                    if c.attrs == {}:
                        content = content + c.text
                        #print(c.text)
                print("內文:", content)

                keyword = []
                k_divs = bs.find("div", class_="keyword page-keyword-area")
                k_strong = k_divs.find_all("strong")
                for k in k_strong:
                    keyword_text = k.text
                    keyword.append(keyword_text)
                print("關鍵字:", keyword)
                
                
                img_p = bs.find("p", style = "text-align: center;")
                if not img_p == None:
                    img_url = img_p.find("img")["src"]
                else:
                    img_url = ""
                print("圖片網址:", img_url)

                j = {"source": "SETN三立新聞網" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep (1)

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: SETN crawler success!')
                crawler_logger.handlers.clear()
        ###SETN爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "setn", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: SETN crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

AppleDaily爬蟲方法

In [15]:
def appledaily_crawler2kafka():
    try:
        ###AppleDaily爬蟲開始###
        for page in range(1, 12):
            time.sleep(2)
            href = "https://tw.news.appledaily.com/politics/realtime/" + str(page)
            res = requests.get(href)
            html = BeautifulSoup(res.text)

            all_news_1 =  html.find_all("ul", class_= "rtddd slvl")
            for all_news in all_news_1:
                news = all_news.find_all("a")
                print("第", page, "頁")
                for n in news:

                    print("新聞來源:", "蘋果日報")

                    #my_news = {}
                    news_url ="https://tw.news.appledaily.com/" + str(n["href"])
                    print("新聞網址:", news_url)

                    #判斷網址是否爬過
                    bf = BloomFilter()
                    # Unicode-objects must be encoded before hashing
                    news_url_u = news_url.encode('utf-8')
                    if bf.isContains(news_url_u):   # 判断字符串是否存在
                        print('url is exists!')
                        continue
                    else:
                        print('url is not exists!')
                        bf.insert(news_url_u)

                    news_per = requests.get(news_url)
                    bs = BeautifulSoup(news_per.text)

                    title = bs.find("h1").text
                    title = re.sub(r"\u3000", " ", title)
                    print("新聞標題:", title)           

                    date = bs.find("div", class_="ndArticle_creat").text.replace("出版時間：", "")
                    print("發佈時間:", date)

                    if not bs.find("div", class_="ndgKeyword") == None:
                        key_word = bs.find("div", class_="ndgKeyword").find_all("a")
                        key_list = []
                        for k in key_word:
                            key_list.append(k.text)
                    else:
                        key_list = []       

                    content_dir = bs.find("div", class_="ndArticle_margin")
                    for a in content_dir.find_all("a"):
                        if not a == None:
                            a.extract()
                    for s in content_dir.find_all("span"):
                        if not s == None:
                            s.extract()
                    for i in content_dir.find_all("iframe"):
                        if not i == None:
                            i.extract()
                    for d in content_dir.find_all("div"):
                        if not d == None:
                            d.extract()
                    for st in content_dir.find_all("style"):
                        if not st == None:
                            st.extract()
                    content = content_dir.text.strip()
                    content = content.replace("）", ")").replace("（", "(").replace("／", "/").replace("╱", "/")       
                    pat = re.compile(r"\(((.{0,15})/.{0,8}[\u5831][\u5c0e])\)")
                    pat_2 = re.compile(r"【((.{0,15})/.{0,8}[\u5831][\u5c0e])】")
                    ans_news = pat.search(content)
                    ans_news_2 = pat_2.search(content)
                    if ans_news == None:
                        author = []
                    elif not ans_news_2 == None:
                        author = ans_news[2].split("、")
                    else:
                        #print(ans_news[2])
                        author = ans_news[2].split("、")
                        content = content.split(ans_news[0])[0]
                    print("作者:", author)   
                    print("內文:", content)  
                    print("關鍵字:", key_list)

                    img_box = bs.find("div", class_="ndAritcle_headPic")
                    if not img_box == None:
                        img_url = img_box.find("img")['src']
                    else:
                        img_url = ""
                    print("圖片網址:", img_url)

                    j = {"source": "蘋果日報" ,"url": news_url, "title": title, "date_": date, "author": author, 
                         "content": content, "kw": key_list, "img_url": img_url}
                    time.sleep(random.randint(1, 2))

                    # TCP連線到logstash
                    crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                    # 爬蟲成功傳送至Elasticsearch的log訊息
                    crawler_logger.info('python-crawler-logstash: AppleDaily crawler success!')
                    crawler_logger.handlers.clear()
        ###AppleDaily爬蟲結束###

                    # 步驟2.指定想要發佈訊息的topic名稱
                    topic_name = "politics_news"

                    print("Now sending messages ...")
                    # 步驟3.產生要發佈到Kafka的訊息
                    # - 參數  # 1: topicName
                    # - 參數  # 2: msgKey
                    # - 參數  # 3: msgValue

                    producer.send(topic = topic_name, key = "appledaily", value = j)
                    print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: AppleDaily crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

NOWnews爬蟲方法

In [16]:
def nownews_crawler2kafka():
    try:
        ###NOWnews爬蟲開始###
        for page in range(1,6):
            href = 'https://www.nownews.com/cat/politics/page/'+str(page)

            response = requests.get(href)
            soup=BeautifulSoup(response.text)
            news = soup.find_all("h3",class_="entry-title td-module-title")

            for n in news:
                
                print("新聞來源:", "NOWnews")

                news_url = n.find("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                response = requests.get(news_url)
                html = BeautifulSoup(response.text)
                title = html.find("h1",class_="entry-title").text
                title = re.sub("\u3000", " ", title)
                print("新聞標題:", title)

                date = html.find("time",class_ = "entry-date").text
                print("發佈時間:", date)

                author = []
                author_text = html.find("div",class_="td-post-author-name").text
                author_text = author_text.split("／")[0]
                author_text = author_text.split("/")[0].replace("記者","").replace("\n","").replace(" ","")
                author.append(author_text)
                print("作者:", author)

                te = html.find("div", class_="td-post-content").contents
                #print(te)
                #print(te[0].name == None)
                content = ""
                for a in te:
                    if a.name == "p":
                        content = content + a.text
                print("內文:", content)

                #處理keyword
                if not html.find("ul",class_="td-tags") == None:
                    kw = html.find("ul",class_="td-tags").find_all("a")
                    keyword = []
                    for k in kw:
                        keyword.append(k.text)
                else:
                    keyword = []
                print("關鍵字:", keyword)

                img_url = html.find("div",class_="td-post-featured-image")
                if not img_url == None:
                    img_url = img_url.find("img")["src"]
                else:
                    img_url = ""
                print("圖片網址:", img_url)

                j = {"source": "NOWnews" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep(random.randint(0,2))

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: NOWnews crawler success!')
                crawler_logger.handlers.clear()
        ###NOWnews爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "nownews", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: NOWnews crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

STORM爬蟲方法

In [17]:
def storm_crawler2kafka():
    try:
        ###STORM爬蟲開始###    
        for page in range(1,5):
            href = 'https://www.storm.mg/category/118/'+str(page)

            response = urlopen(href)
            html = BeautifulSoup(response)
            #找到特定範圍在選取網址列
            newf = html.find("div",class_ = "middle_category_cards")
            #選取所有單篇網址列
            news = newf.find_all("div",class_ = "category_card")
            #迴圈抓取整頁新聞連結
            for r in news:

                print("新聞來源:", "風傳媒 THE STORM MEDIA")

                news_url = r.find("a",class_ = "card_link")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                response = urlopen(news_url)
                html = BeautifulSoup(response)

                title = html.find("h1", id = "article_title").text
                print("新聞標題:", title)

                date = html.find("span", class_="info_time").text
                print("發佈時間:", date)

                author = []
                author_text = html.find("span", class_="info_author").text.replace("新新聞","")
                author.append(author_text)
                print("作者:", author)

                content = html.find("div", id="CMS_wrapper").text.replace("\n","").replace("\t","")
                print("內文:", content)

                keyword = html.find("div", id="tags_list_wrapepr").text.split()
                print("關鍵字:", keyword)

                img_url = html.find("img", id="feature_img")
                if not img_url == None:
                    img_url = img_url["src"]
                else:
                    img_url = ""
                print("圖片網址:", img_url)
                
                j = {"source": "風傳媒 THE STORM MEDIA" ,"url": news_url, "title": title, "date_": date, "author": author, 
                     "content": content, "kw": keyword, "img_url": img_url}
                time.sleep(random.randint(0,2))

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: STORM crawler success!')
                crawler_logger.handlers.clear()
        ###STORM爬蟲結束###
    
                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "storm", value = j)
                print("Message has finished sending!", "\n")
            
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: STORM crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

ETtoday爬蟲方法

In [18]:
def ettoday_crawler2kafka():
    try:
        ###ETtoday爬蟲開始###
        for page in range(1, 1007):
            url = "https://www.ettoday.net/news_search/doSearch.php?keywords=%E6%94%BF%E6%B2%BB&kind=1" + "&idx=1&page=" + str(
                page)
            print("第", page, "頁"," ","Processing: ", url)
            try:
                response = urlopen(url)
            except HTTPError:
                print("大概是結束了")
                break
            html = BeautifulSoup(response)

            all_ar = html.find("div", class_="result_archive")

            val = all_ar.find_all("div", class_="archive clearfix")

            for u in val:

                print("新聞來源:", "ETtoday")

                box2 = u.find("div", class_="box_2")
                news_url = box2.find("a")["href"]
                print("新聞網址:", news_url)


                #判斷網址是否爬過
                bf = BloomFilter()
                # Unicode-objects must be encoded before hashing
                news_url_u = news_url.encode('utf-8')
                if bf.isContains(news_url_u):   # 判断字符串是否存在
                    print('url is exists!')
                    continue
                else:
                    print('url is not exists!')
                    bf.insert(news_url_u)


                a_response = urlopen(news_url)
                a_html = BeautifulSoup(a_response, "html.parser")

                title = a_html.find("h1", class_="title").text
                title = re.sub("\u3000", " ", title)
                print("新聞標題:", title)

                date = a_html.find("time", class_="date").text.strip()
                date = re.sub("年", "/", date)
                date = re.sub("月", "/", date)
                date = re.sub("日", "", date)
                print("發佈時間:", date)


                a_story = a_html.find("div", class_="story")                
                content_p = a_story.find_all("p")
                author_c = ""
                for c_p in content_p:
                    #如果c_p裡面的第一個格子是None
                    if c_p.contents[0].name == None:
                        author_c = author_c + c_p.text
                    elif c_p.contents[0].name == "span":
                        author_c = author_c + c_p.text
                author = []
                if re.search(r".*／.?.?報導", author_c) == None:
                    author = []
                else:
                    #因為作者與內文會放在同一個格子，所以將內文中會抓到的作者部分在內文處理時刪除
                    content_del = re.search(r".*／.?.?報導", author_c).group()
                    author_text = re.search(r".*／.?.?報導", author_c).group() 
                author.append(author_text)
                print("作者:", author)              

                content = ""
                for c in content_p:
                    if c.contents[0].name == None:
                        content = content + c.text
                        content = content.replace("\u3000", " ")      
                        content = content.replace(content_del, "")            
                print("內文:", content)

                keyword = []
                a_web = a_html.find("article")
                a_kw = a_web.find("p", class_="tag")
                kw = a_kw.find_all("a", target="_blank")
                for k in kw:
                    kw_n = k.text
                    keyword.append(kw_n)
                if keyword == [""]:
                    keyword = []
                print("關鍵字:", keyword)

                if not a_story.find("img")['src'] == None:
                    img_url = a_story.find("img")['src']
                    img_url = "https:" + img_url
                else:
                    img_url = ""
                print("圖片網址:", img_url)

                j = {"source": "ETtoday" ,"url": news_url, "title": title, "date_": date, "author": author, "content": content, 
                     "kw": keyword, "img_url": img_url}
                time.sleep(1)

                # TCP連線到logstash
                crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                crawler_logger.info('python-crawler-logstash: ETtoday crawler success!')
                crawler_logger.handlers.clear()
        ###ETtoday爬蟲結束###

                # 步驟2.指定想要發佈訊息的topic名稱
                topic_name = "politics_news"

                print("Now sending messages ...")
                # 步驟3.產生要發佈到Kafka的訊息
                # - 參數  # 1: topicName
                # - 參數  # 2: msgKey
                # - 參數  # 3: msgValue

                producer.send(topic = topic_name, key = "ettoday", value = j)
                print("Message has finished sending!", "\n")
    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
        # TCP連線到logstash
        crawler_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
        # 爬蟲失敗傳送至Elasticsearch的log訊息
        crawler_logger.error('python-crawler-logstash: ETtoday crawler error message!! [' + news_url + ']')
        crawler_logger.handlers.clear()

所有爬蟲執行方法

In [19]:
def all_crawler():
    tvbs_crawler2kafka()
    setn_crawler2kafka()
    appledaily_crawler2kafka()
    nownews_crawler2kafka()
    storm_crawler2kafka()
    ettoday_crawler2kafka()

In [21]:
ettoday_crawler2kafka()

第 1 頁   Processing:  https://www.ettoday.net/news_search/doSearch.php?keywords=%E6%94%BF%E6%B2%BB&kind=1&idx=1&page=1
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181229/1343337.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181229/1343157.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1343152.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1343184.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1343153.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1342945.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1343033.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1342747.htm
url is exists!
新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1342917.htm
url is not exists!
新聞標題: 淡海輕軌不打烊！跨年夜通宵營運39.5小時
發佈時間: 2018/12/28 15:33
作者: ['政治中心／綜合報導']
內文: 為了讓民眾好好跨年，淡海輕軌在12月31日跨年夜通霄營運，連續39.5小時不打烊

Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1342682.htm
url is not exists!
新聞標題: 8千張春聯被搶光！高嘉瑜嚇壞 加碼再送「收藏用」電子檔
發佈時間: 2018/12/28 11:53
作者: ['政治中心／綜合報導']
內文: 被封為「港湖吸票機」的民進黨台北市議員高嘉瑜高票連任，人氣水漲船高，日前公布「抱豬甜笑款」及「眼球運動款」的春聯再讓網友暴動，短短3小時就送出8000份，讓高嘉瑜坦言同仁都嚇到吃手手，並說目前已經沒辦法再追加提供寄送，但仍開放部分自取，她也會在服務處發送，另外上傳電子檔，讓想要收藏的民眾下載。高嘉瑜日前在臉書上傳4張照片，身穿黃色七分袖V領、綁著招牌公主頭，對鏡頭嘟嘴微笑，除了學台北市長柯文哲的招牌「抓頭」姿勢外，更有選前在台北市長落選人姚文智造勢晚會備受討論的「眼球運動」，當時還詢問網友喜歡哪張照片放春聯，最後則選定「抱豬甜笑款」及「眼球運動款」。超萌春聯樣品照一曝光馬上引來網友瘋搶，高嘉瑜表示，往年外縣市春聯大概準備6、700份，今年3小時內就有8000份要寄送，讓辦公室同仁嚇壞，希望外縣市的朋友見諒，現在已經沒辦法再追加提供寄送，春聯印好公告後可以在她的服務處或里長辦公室索取，自己也會親自發送，但數量有限，發完為止。不過，高嘉瑜仍貼心上傳春聯電子檔，提供給想收藏的民眾，不少網友則喊，「我想去你的服務處拿100張」、「太夯啦♥」、「真香，啊不是 XDDDD」。
關鍵字: ['高嘉瑜', '民進黨', '姚文智']
圖片網址: https://cdn2.ettoday.net/images/3795/3795273.jpg
Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181228/1342497.htm
url is not exists!
新聞標題: 童仲彥召喚嫩妹小編 網友一看萌照暴動求上車
發佈時間: 2018/12/28 11:11
作者: ['

Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1342324.htm
url is not exists!
新聞標題: 高嘉瑜「眼球運動」春聯加碼1萬張秒殺了！ 網友哀嚎：快出Line貼圖
發佈時間: 2018/12/27 22:26
作者: ['政治中心／綜合報導']
內文: 被封為港湖女神的台北市議員高嘉瑜，27日在臉書宣布推出「眼球運動」春聯，吸引粉絲瘋搶後她再加碼1萬張，結果不到3個小時8千張春聯已經秒殺，搶不到春聯的網友留言要高嘉瑜「快出Line貼圖」。高嘉瑜推出「抱豬甜笑款」及「眼球運動款」春聯，提供給想要的粉絲索取，她也貼心提醒網友，數量有限很可能會秒殺，沒想到一個小時過後，春聯真的被搶光了，於是她再加碼到1萬份，不到3個小時，吸引2千粉絲瘋搶8千張春聯。高嘉瑜開心在臉書說，謝謝大家喜歡，也提醒大家春聯印好會在公告，不提供寄送服務，要到服務處親領。搶不到春聯的網友留言表示，「快點出賴貼圖」、「這太秒殺了吧～連車尾燈都看不到」、「我沒登記到」、「果然是港湖最高票」。
關鍵字: ['高嘉瑜']
圖片網址: https://cdn2.ettoday.net/images/3783/d3783932.jpg
Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1341926.htm
url is not exists!
新聞標題: 謝長廷歲末盤點「駐日5大政績」 網友嘴翻：2020酸宗痛
發佈時間: 2018/12/27 22:16
作者: ['政治中心／綜合報導']
內文: 駐日代表謝長廷因關西機場造謠案，飽受外界批評，他27日晚間在臉書再貼文，盤點一下過去一年來的台日關係，強調來日觀光訪問的人數已經高達480萬人，二年多來成長一百萬人左右，「指責駐日工作沒有績效，顯然與事實不符。」謝長廷列出五項成績，如下：謝長廷表示，「以上小小成績當然是代表處全體同仁努力的結果，但指責駐日工作沒有績效，顯然與事實不符

Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1341967.htm
url is not exists!
新聞標題: 包庇謝長廷？網友辱大阪辦事處「爛到該死」 李明賢質疑警方吃案
發佈時間: 2018/12/27 15:22
作者: ['記者蔣婕妤／台北報導']
內文: 前駐大阪辦事處長蘇啟誠自殺事件持續延燒，國民黨台北市議員李明賢27透過新聞稿指控，綠營將事件導向與假新聞有關，警方火速移送一名游姓學生，但另一篇署名idcc替駐日代表謝長廷辯護、並怒批大阪代表處「爛到該死」的煽動文章，警方雖受理偵辦卻草草結案，李明賢痛批是「技術性吃案」，懷疑背後有高層政治力量干預，導致警方不敢繼續查辦下去。今年9月「燕子颱風」肆虐日本，造成許多台灣人滯留日本，署名guruguru的網友在PTT發文，指「中國外交使館積極派車接送中國遊客、但台灣駐日單位卻無作為」，經謝長廷11月間告發後，1個月內警方以違反《社會秩序維護法》，移送南投地方法院，並查出guruguru的身分，綠營也強調假新聞才是蘇啟誠尋短的原因。李明賢指出，風災期間另一名「idcc」網友在PTT上發文指稱，大阪辦事處不歸謝長廷所管，還說大阪辦事處的人員「爛到不行、爛到該死」，甚至是「黨國餘孽」，李明賢與新北市議員葉元之在9月中前往台北市刑大舉發違反《社會秩序維護法》。李明賢表示，警方偵辦結果指出「idcc」本人已刪除原文，加上PTT不願提供相關原始資料與歷次登入紀錄，因此以「證據能力有疑慮」為由結案，他不滿指控，明明有網友肉搜到「idcc」曾於2004年發文身分疑似是政大碩一，甚至使用ip是來自政府官方，但警方卻指稱IP紀錄查閱須符合通保法規範，電信紀錄僅保存6個月，「態度消極到令人匪夷所思！」李明賢痛批，同樣是用《社會秩序維護法》偵辦，警方查辦GURUGURU就能立即查出身分並移送法院，但替謝長廷辯護的「idcc」，警方辦案態度推拖拉，他嗆，「不免令人懷疑背後有高層政治干預，才會導致一樣案件兩種結果，是否背後特定力量仍在袒護包庇謝長廷？」
關鍵字: ['謝長廷']
圖片網址: https://cdn2.ettoday.net/image

Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1341679.htm
url is not exists!
新聞標題: 高志鵬將提非常上訴 「助理索賄200萬卻是老闆坐牢！」
發佈時間: 2018/12/27 11:30
作者: ['記者翁嫆琄／台北報導']
內文: 民進黨前立委高志鵬捲入貪污案，遭判刑4年半、褫奪公權4年定讞。高志鵬27日出面強調自己難以接受，此案是他「一時用人不當」，前助理私下索賄200萬，事後則以政治獻金名義將50萬交給辦公室，事發後該助理卻轉為污點證人，控他貪污，「如果是我藉民眾陳情的機會，要助理去向廠商期約報酬200萬元，何以助理拿了150萬元，卻只給我50萬元？」「助理索賄，老闆坐牢！」高志鵬說，昨天看到最高法院發佈新聞稿，駁回貪污案件上訴，仍維持二審有罪判決，判處四年半的有期徒刑。身為法律人對於司法判決，他會選擇面對不會逃避，但對於這樣的判決結果，實難以接受。他身為立委為服務選民，對於民眾陳情案本應盡可能提供協助，但有誰想到一時用人不當，助理竟會利用陳情案件，私下與廠商約定報酬，事後以政治獻金名義將其中一小部分金額交給選舉期間的辦公室會計人員，於東窗事發後轉為污點證人指稱他「對於職務上行為，要求、期約或收受賄賂或其他不正利益」涉有貪污。他指出，後來法院改以「對於非主管或監督之事務，明知違背法令，利用職權機會或身分圖自己或其他私人之不法利益，因而獲得利益者。」的圖利罪判刑。整件事實的過程，宛如一場閙劇，助理要了兩百萬酬勞，只給他50萬元，當時他還開玩笑說，未來如果要應徵助理，應該很多人應徵，拿那麼多錢，出了事還可以推給老闆，「助理索賄，老闆坐牢」。但經由司法審判後，卻落得貪污罪定讞的結果。高志鵬強調，從頭到他跟對方也不認識，沒見面、沒談過，開協調會也只是請官員來，讓當事人有機會溝通。他也點出此案3項不合理之處，首先，若是他藉民眾陳情的機會，要助理去向廠商期約報酬200萬元，為何助理拿了150萬元，卻只給他50萬元？第二，若是他要助理去向廠商收取期約報酬200萬元，為何助理收了錢後，不立即將賄款交給他，而是等到選舉期間需要籌措選舉經費時，該助理才以「政治獻金」

Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1341472.htm
url is not exists!
新聞標題: 針對總統大選的初選機制 孫大千想問黨中央6個問題
發佈時間: 2018/12/27 08:59
作者: ['政治中心／綜合報導']
內文: 前新北市長朱立倫盼國民黨黨內有公開透明機制，推派出最佳總統候選人。對此，前國民黨立委孫大千27日針對總統大選的初選機制，在臉書提出6個問題。朱立倫24日卸下新北市長一職，立即宣布將成立工作室，投入2020年總統選舉，朱立倫的幕僚透露，朱立倫遵從國民黨內初選機制，搶先表態以呼應民眾的期待，主要是希望國民黨不要私下喬人選，盼黨內有公開透明機制，產生最佳總統候選人。對此，針對總統大選的初選機制，孫大千想問國民黨黨中央6個問題，第一，2020年的總統大選對國民黨來說，是不是最重要的一場選戰？第二，黨內高層早已心知肚明，2020年的總統和立委不可能分開選舉，對不對？孫大千質問，第三，為什麼遲至今日，黨中央仍然不願意針對總統大選的初選制度，提出一套清楚明白的遊戲規則呢？
第四，這樣的拖拖拉拉，曖昧不明，是否是要為了特定人士保留運作的彈性空間？孫大千反問，第五，身為有志於領導台灣的未來，解決人民的困境的最大在野黨，為什麼不能展現出「大開大闔，大公無私」的氣魄？第六，國民黨究竟是在怕什麼？
關鍵字: ['總統大選', '孫大千', '國民黨', '朱立倫']
圖片網址: https://cdn2.ettoday.net/images/3787/d3787333.jpg
Now sending messages ...
Message has finished sending! 

新聞來源: ETtoday
新聞網址: https://www.ettoday.net/news/20181227/1341466.htm
url is not exists!
新聞標題: 柯文哲、韓國瑜六都市長合體 27日首度共同出席行政院會
發佈時間: 2018/12/27 08:23
作者: ['政治中心／綜合報導']
內文: 台北市長柯文哲、新北市長侯友宜、桃園

KeyboardInterrupt: 

主程式

In [23]:
#!pip install schedule
import schedule
import time
import datetime
import threading

schedule.every(10).minutes.do(all_crawler)
#def job():
#    print("I'm working...")
#schedule.every(1).minutes.do(job)
#schedule.every().day.at("16:42").do(tvbs_crawler2kafka)
#schedule.every().day.at("17:51").do(setn_crawler2kafka)
#schedule.every().day.at("22:27").do(appledaily_crawler2kafka) 
#schedule.every().day.at("22:27").do(nownews_crawler2kafka)
#schedule.every().day.at("22:27").do(storm_crawler2kafka())

while True:
    schedule.run_pending()
    time.sleep(1)


In [ ]:
# coding=utf-8
import threading, queue, time, urllib
from  urllib import request
baseUrl = 'http://www.pythontab.com/html/pythonjichu/'
urlQueue = queue.Queue()
for i in range(2, 10):
    url = baseUrl + str(i) + '.html'
    urlQueue.put(url)
    #print(url)
def fetchUrl(urlQueue):
    while True:
        try:
            #不阻塞的读取队列数据
            url = urlQueue.get_nowait()
            i = urlQueue.qsize()
        except Exception as e:
            break
        print ('Current Thread Name %s, Url: %s ' % (threading.currentThread().name, url))
        try:
            response = urllib.request.urlopen(url)
            responseCode = response.getcode()
        except Exception as e:
            continue
        if responseCode == 200:
            #抓取内容的数据处理可以放到这里
            #为了突出效果， 设置延时
            time.sleep(1)
if __name__ == '__main__':
    startTime = time.time()
    threads = []
    # 可以调节线程数， 进而控制抓取速度
    threadNum = 2
    for i in range(0, threadNum):
        t = threading.Thread(target=fetchUrl, args=(urlQueue,))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        #多线程多join的情况下，依次执行各线程的join方法, 这样可以确保主线程最后退出， 且各个线程间没有阻塞
        t.join()
    endTime = time.time()
    print ('Done, Time cost: %s ' %  (endTime - startTime))